### Asking information about ChiLit books to ChatGPT

In [1]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.load.dump import dumps
from pydantic import BaseModel, Field
from pydantic.config import ConfigDict
from typing import Annotated, Literal
from enum import Enum
from typing import Literal
from typing import List
import pandas as pd
import json
import os
import time
from dotenv import load_dotenv

In [2]:
if not load_dotenv():
    os.environ['OPENAI_API_KEY'] = input("OpenAI API key: ")  

In [3]:
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv('OPENAI_API_KEY')
)

In [4]:
model.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-f483f726-8191-45ec-8598-1f1057df8eb9-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
class Book(BaseModel):
    plot : str
    characters : str
    genre : str
    style : str
    themes : str

In [4]:
base_prompt = """I'd like to have some information about the book {book}.
Please provide in output:

1) a summary of the plot
2) a brief description of the main characters
3) an overview of the genre, the style and narrative
4) an overview the style and narrative
5) a description of the main themes

"""

In [5]:
def get_prompt(base_prompt, parser):

    template = base_prompt + "\n" + "{format_instructions}'"

    prompt = PromptTemplate(
        template = template,
        input_variables = ["book"],
        partial_variables = {"format_instructions": parser.get_format_instructions()}
    )

    return prompt

In [7]:
parser = PydanticOutputParser(pydantic_object = Book)
prompt = get_prompt(base_prompt, parser)
chain = prompt | model | parser

In [8]:
# Test
result = chain.invoke({"book": "The Carved Lions by Molesworth, Mary Louisa"})

In [9]:
result

Book(plot='The Carved Lions follows the story of a young girl named Mary, who is sent to live with her aunt and uncle in a large, mysterious house. The house is filled with strange carvings of lions, which seem to hold secrets of their own. As Mary explores her new surroundings, she uncovers the history of the house and the significance of the carved lions, leading her to a deeper understanding of her family and herself. The narrative intertwines elements of adventure, mystery, and personal growth as Mary navigates her relationships and the challenges she faces.', characters="Mary is the protagonist, a curious and brave young girl. Her aunt and uncle, who are somewhat aloof, provide a contrast to her adventurous spirit. Other characters include the enigmatic housekeeper, who knows more about the house's secrets than she lets on, and various townsfolk who contribute to the unfolding mystery surrounding the carved lions.", genre="The Carved Lions is primarily a children's adventure novel

In [2]:
result_file = './data/ChiLit_Info.csv'
df = pd.DataFrame(columns = ['shorttitle', 'title','author','date','plot','characters', 'genre', 'style', 'themes'])
df.to_csv(result_file, encoding = "utf-8", index = False)

In [13]:
df_corpus = pd.read_csv("./data/ChiLit_Metadata.csv", encoding = "utf-8")

In [4]:
df_corpus.head()

,author,title,date,shorttitle
0,"Anstey, F.",The Brass Bottle,1900,brass
1,"Crockett, S. R.",The Surprising Adventures of Sir Toady Lion wi...,1897,toadylion
2,"Mare, W. De la",The Three Mulla-mulgars,1910,mulgars
3,"Ewing, J. H. G.",Mrs. Overtheway's Remembrances,1869,overtheway
4,"Ewing, J. H. G.",Jackanapes,1883,jackanapes


In [ ]:
for index, book in df_corpus.iterrows():
    print(f"{book['title']} written by {book['author']}")
    
    try:
        result = chain.invoke({"book": f"{book['title']} written by {book['author']}"})
        new_row = {
            'shorttitle': book['shorttitle'],
            'title': book['title'],
            'author': book['author'],
            'date': book['date'],
            'plot': result.plot,
            'characters': result.characters,
            'genre': result.genre,
            'style': result.style,
            'themes': result.themes
        }
        df = pd.DataFrame([new_row])
        df.to_csv(result_file, encoding = "utf-8", mode='a', index=False, header=False)
    except:
        print("Error: ", expression)
   

### Themes

In [14]:
df_info = pd.read_csv("./data/ChiLit_Info.csv", encoding = "utf-8")

In [11]:
df_info.head()

,shorttitle,title,author,date,plot,characters,genre,style,themes
0,brass,The Brass Bottle,"Anstey, F.",1900,The Brass Bottle follows the story of a young ...,"Felix H. M. H. is the protagonist, a young arc...",The Brass Bottle is primarily a fantasy novel ...,The narrative style of The Brass Bottle is cha...,Key themes in The Brass Bottle include the exp...
1,toadylion,The Surprising Adventures of Sir Toady Lion wi...,"Crockett, S. R.",1897,The Surprising Adventures of Sir Toady Lion wi...,"Sir Toady Lion is the main protagonist, charac...",The book falls into the genre of children's li...,The narrative style of the book is characteriz...,"Key themes in the book include friendship, bra..."
2,mulgars,The Three Mulla-mulgars,"Mare, W. De la",1910,The Three Mulla-mulgars follows the adventures...,The main characters are the three Mulla-mulgar...,The book is primarily a children's fantasy adv...,The narrative style is characterized by a play...,Key themes in The Three Mulla-mulgars include ...
3,overtheway,Mrs. Overtheway's Remembrances,"Ewing, J. H. G.",1869,Mrs. Overtheway's Remembrances is a collection...,"The main character is Mrs. Overtheway herself,...",The book falls into the genre of literary fict...,The narrative style is reflective and conversa...,Key themes in the book include the passage of ...
4,jackanapes,Jackanapes,"Ewing, J. H. G.",1883,Jackanapes is a heartwarming story about a you...,"The main characters include Jack, a kind-heart...","Children's literature, Adventure",The style of 'Jackanapes' is characterized by ...,The main themes of 'Jackanapes' include friend...


In [17]:
for index, book in df_info.iterrows():
    print('\n---------------------------')
    print(f"{book['title']} ({book['shorttitle']})")
    print(book['themes'])


---------------------------
The Brass Bottle (brass)
Key themes in The Brass Bottle include the exploration of desire and the consequences of one's wishes, the nature of freedom and control, and the importance of human relationships. The story also touches on the idea of self-discovery and the realization that true happiness often comes from within rather than from external sources.

---------------------------
The Surprising Adventures of Sir Toady Lion with Those of General Napoleon Smith (toadylion)
Key themes in the book include friendship, bravery, and the importance of teamwork. The adventures of Sir Toady Lion and General Napoleon Smith highlight the value of loyalty and support in overcoming obstacles. Additionally, the story explores the idea of courage in the face of adversity and the joy of embarking on adventures with friends.

---------------------------
The Three Mulla-mulgars (mulgars)
Key themes in The Three Mulla-mulgars include friendship, the quest for freedom, the 

### Genres

In [18]:
for index, book in df_info.iterrows():
    print('\n---------------------------')
    print(f"{book['title']} ({book['shorttitle']})")
    print(book['genre'])



---------------------------
The Brass Bottle (brass)
The Brass Bottle is primarily a fantasy novel with elements of comedy and adventure. It blends magical realism with humorous situations, making it an entertaining read that appeals to a wide audience.

---------------------------
The Surprising Adventures of Sir Toady Lion with Those of General Napoleon Smith (toadylion)
The book falls into the genre of children's literature, specifically adventure and fantasy. It combines elements of whimsy and humor, making it appealing to younger audiences while also containing deeper themes that resonate with older readers.

---------------------------
The Three Mulla-mulgars (mulgars)
The book is primarily a children's fantasy adventure, blending elements of animal fiction with moral lessons and whimsical storytelling.

---------------------------
Mrs. Overtheway's Remembrances (overtheway)
The book falls into the genre of literary fiction, with elements of memoir and nostalgia. It captures the

### Plots

In [19]:
for index, book in df_info.iterrows():
    print('\n---------------------------')
    print(f"{book['title']} ({book['shorttitle']})")
    print(book['plot'])


---------------------------
The Brass Bottle (brass)
The Brass Bottle follows the story of a young architect named Felix H. M. H. who discovers a mysterious brass bottle that contains a genie named Balthazar. When Felix accidentally releases Balthazar, he is granted wishes that lead to a series of humorous and chaotic events. As Felix navigates the consequences of his wishes, he learns about the complexities of desire, responsibility, and the nature of happiness. The story unfolds with Felix's attempts to control the genie and the ensuing misadventures that arise from his wishes, ultimately leading to a resolution that emphasizes the importance of personal agency and the value of human relationships.

---------------------------
The Surprising Adventures of Sir Toady Lion with Those of General Napoleon Smith (toadylion)
The Surprising Adventures of Sir Toady Lion with Those of General Napoleon Smith follows the whimsical escapades of Sir Toady Lion, a brave and noble lion, and General